In [95]:
import numpy as np
import cupy as cp
import asyncio
import hashlib

class API:
    def __init__(self, size, int_range=(1, 10)):
        self.matrices = []
        self.matrices.append(
            np.random.randint(int_range[0], int_range[1], (size, size))
        )
        self.matrices.append(
            np.random.randint(int_range[0], int_range[1], (size, size))
        )
        self.count = 0
        self.analytics: list[tuple[int, int, int]] = []

    def clear(self):
        self.count = 0

    async def call(self, endpoint):
        self.count += 1
        endpoint = endpoint.split("/")
        matrix = int(endpoint[1])
        type = endpoint[2]
        index = int(endpoint[3])
        
        await asyncio.sleep(0.01)

        return self.getRow(matrix, index) if type == "row" else self.getCol(matrix, index)

    def getRow(self, matrix, row):
        return self.matrices[matrix][row, :]

    def getCol(self, matrix, col):
        return self.matrices[matrix][:, col]
    
    def getHash(self):
        answer = np.dot(self.matrices[0], self.matrices[1])
        flattened = answer.flatten()
        joined = "".join([str(x) for x in flattened])
        return hashlib.sha256(joined.encode()).hexdigest()


In [96]:
api = API(2)
api.getHash()

'246148129'

In [97]:
def calcHash(matrix):
    flattened = matrix.flatten()
    joined = "".join([str(x) for x in flattened])
    # return hashlib.sha256(joined.encode()).hexdigest()
    return joined

In [98]:
async def fetch_compute(api, size):
    
    result = cp.zeros((size, size), dtype=cp.int32)
    row_tasks = [asyncio.ensure_future(api.call(f"/0/row/{i}")) for i in range(size)]
    col_tasks = [asyncio.ensure_future(api.call(f"/1/col/{i}")) for i in range(size)]

    rows = await asyncio.gather(*row_tasks)
    cols = await asyncio.gather(*col_tasks)

    # Converting rows and cols to CuPy arrays
    rows = cp.array(rows)
    cols = cp.array(cols)

    compute_tasks = []

    for i in range(size):
        for j in range(size):
            task = asyncio.ensure_future(compute_cell(rows[i], cols[j], i, j, result))
            compute_tasks.append(task)

    await asyncio.gather(*compute_tasks)
    return result

async def compute_cell(a, b, col, row, result):
    print(f"Computing cell {row}, {col}: {a} * {b}")
    result[row, col] = cp.asarray(int(cp.dot(a, b)))

size = 2
result = await fetch_compute(api, size)
print(cp.asnumpy(result))
print(calcHash(cp.asnumpy(result)))
print(api.getHash())
print(api.count)

api.clear()

Computing cell 0, 0: [2 5] * [2 4]
Computing cell 1, 0: [2 5] * [8 9]
Computing cell 0, 1: [6 9] * [2 4]
Computing cell 1, 1: [6 9] * [8 9]
[[ 24  48]
 [ 61 129]]
244861129
246148129
4


In [99]:
api.matrices

validation = np.dot(api.matrices[0], api.matrices[1])
hash(validation.tobytes())

-3281430838408129304

In [100]:
print(api.count)

0


In [101]:
result_hash = hash(result.tobytes())
print(result_hash)

print(api.hash)

-8152908661501143930


AttributeError: 'API' object has no attribute 'hash'

In [ ]:
test = np.random.rand(3, 3)
test

array([[0.85307433, 0.86669397, 0.65647558],
       [0.01481618, 0.5762393 , 0.35083939],
       [0.42792569, 0.34421946, 0.58111142]])

In [ ]:
z = np.array([[0.5, 0.2],[0.3, 0.7]])
x= np.array([[0.6, 0.4], [0.1, 0.9]])
np.dot(z, x)
# np.dot(z, x).tobytes()
# hash(np.dot(z, x).tobytes())

np.dot(z[0, :], x[:, 0])
np.dot(z[0, :], x[:, 1])
np.dot(z[1, :], x[:, 0])

0.25